In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(100,1000), dtype=np.float64) 
sum_resource=np.zeros(shape=(1,1000), dtype=np.float64)
sum_cost=np.zeros(shape=(1,1000), dtype=np.float64)
iteration_update = 0.01
A=np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt") #网络拓扑的邻接矩阵 对角元素要为0
D=np.zeros(shape=(100,100))# 网络的度矩阵
L=np.zeros(shape=(100,100))# 网络的拉普拉斯矩阵 L=D-A
W=np.zeros(shape=(100,100)) #可以直接采用网络对应的拉普拉斯矩阵  因为无向图的拉普拉斯矩阵天然满足行和和列和为0
iteration=[]
a=np.loadtxt("100_uniform_distribution_(1,2)_point1.txt")
b=np.loadtxt("100_normal_distribution_(2,0.6)_point1.txt")
agent_gradient=np.zeros(shape=(100,1000), dtype=np.float64) 
bias=np.loadtxt("1000_Gaussian_distribution_(500,10).txt")
C=np.zeros(shape=(1,1000), dtype=np.float64)#adaptive robust clipping的裁剪参数 
#设置为全局第b大的梯度信息 b为网络中设备损坏的用户上界 每一次迭代就需要计算一次新的C值

In [3]:
#根据邻接矩阵生成网络对应的度矩阵
for i in range(0,100):
    d=0
    for j in range(0,100):
        if A[i][j]==1:
            d=d+1
    D[i][i]=d

#根据邻接矩阵和度矩阵计算拉普拉斯矩阵
L=D-A

#选择用拉普拉斯矩阵作为聚合矩阵
W=L
    
#资源量的初始化 需要给定可行的初始化 即满足平均资源约束
for i in range (0,100):
    agent_resource_quantity[i][0]=50
    
#资源量总和 初始化
sum=0
for i in range (0,100):
    sum=sum+agent_resource_quantity[i][0]
sum_resource[0]=sum

iteration.append(0)

#初始梯度计算
for i in range (0,100):
    if i==7 or i==20 or i==34:
        agent_gradient[i][0]=2*a[i]*agent_resource_quantity[i][0]-2*a[i]*b[i]+bias[0]
    else:
        agent_gradient[i][0]=2*a[i]*agent_resource_quantity[i][0]-2*a[i]*b[i]
    
#计算初始的C值
column_data = agent_gradient[:, 0]
forth_largest = np.partition(np.abs(column_data), -4)[-4]
C[0][0]=forth_largest

#诚实用户成本总和初始化
sum=0
for i in range(0,100):
    sum=sum+a[i]*(agent_resource_quantity[i][0]-b[i])*(agent_resource_quantity[i][0]-b[i])
sum_cost[0][0]=sum

In [4]:
#基于初始化的权重梯度下降无中心资源分配算法
def IWDGD_Gaussian_bias_C(a,b,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias,C):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************IWDGD算法+计算误差+鲁棒****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量总和为:",sum_resource[0][0])
    print("*************************")
    print("初始成本总和为:",sum_cost[0][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,1000):#迭代次数
        iteration.append(k)
 
        #所有用户资源申请量基于可行初始化权重梯度下降的迭代  假设部分用户因为设备损坏 导致梯度的计算有误差     
        for i in range (0,100):
            weight_gradient_sum = np.float64(0)
            for j in range (0,100):  
                if np.abs(agent_gradient[j][k-1])>C[0][k-1]:
                    weight_gradient_sum=weight_gradient_sum+W[i][j]*(C[0][k-1]/np.abs(agent_gradient[j][k-1])*agent_gradient[j][k-1])
                else:
                    weight_gradient_sum=weight_gradient_sum+W[i][j]*agent_gradient[j][k-1]                   
                #weight_gradient_sum = np.round(weight_gradient_sum, decimals=3)
            agent_resource_quantity[i][k]=agent_resource_quantity[i][k-1]-iteration_update*weight_gradient_sum
                #agent_resource_quantity[i][k] = np.round(agent_resource_quantity[i][k], decimals=3)                      
        
        #计算所有用户的资源量总和
        sum1=np.float64(0)
        for i in range (0,100):
            sum1 = sum1 + agent_resource_quantity[i][k]
        sum_resource[0][k]=sum1
        
        #计算所有用户的成本总和
        sum2=np.float64(0)
        for i in range(0,100):
            sum2=sum2+a[i]*(agent_resource_quantity[i][k]-b[i])*(agent_resource_quantity[i][k]-b[i])
        sum_cost[0][k]=sum2
        
        #计算当前迭代的用户梯度
        for i in range (0,100): 
            if i==7 or i==20 or i==34:
                agent_gradient[i][k]=2*a[i]*agent_resource_quantity[i][k]-2*a[i]*b[i]+bias[k]
            else:
                agent_gradient[i][k]=2*a[i]*agent_resource_quantity[i][k]-2*a[i]*b[i]
                
        #计算当前迭代的C值
        column_data = agent_gradient[:, k]
        forth_largest = np.partition(np.abs(column_data), -4)[-4]
        C[0][k]=forth_largest
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%50==0:
        #if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量总和为:",sum_resource[0][k])
            print("****************************************************************************")
            print("成本总和为:",sum_cost[0][k])
            print("****************************************************************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=1/(pow(k+1,0.05)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
IWDGD_Gaussian_bias_C(a,b,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias,C)

****************************IWDGD算法+计算误差+鲁棒****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 50.0
agent 2 的初始资源申请量为: 50.0
agent 3 的初始资源申请量为: 50.0
agent 4 的初始资源申请量为: 50.0
agent 5 的初始资源申请量为: 50.0
agent 6 的初始资源申请量为: 50.0
agent 7 的初始资源申请量为: 50.0
agent 8 的初始资源申请量为: 50.0
agent 9 的初始资源申请量为: 50.0
agent 10 的初始资源申请量为: 50.0
agent 11 的初始资源申请量为: 50.0
agent 12 的初始资源申请量为: 50.0
agent 13 的初始资源申请量为: 50.0
agent 14 的初始资源申请量为: 50.0
agent 15 的初始资源申请量为: 50.0
agent 16 的初始资源申请量为: 50.0
agent 17 的初始资源申请量为: 50.0
agent 18 的初始资源申请量为: 50.0
agent 19 的初始资源申请量为: 50.0
agent 20 的初始资源申请量为: 50.0
agent 21 的初始资源申请量为: 50.0
agent 22 的初始资源申请量为: 50.0
agent 23 的初始资源申请量为: 50.0
agent 24 的初始资源申请量为: 50.0
agent 25 的初始资源申请量为: 50.0
agent 26 的初始资源申请量为: 50.0
agent 27 的初始资源申请量为: 50.0
agent 28 的初始资源申请量为: 50.0
agent 29 的初始资源申请量为: 50.0
agent 30 的初始资源申请量为: 50.0
agent 31 的初始资源申请量为: 50.0
agent 32 的初始资源申请量为: 50.0
agent 33 的初始资源申请量为: 50.0
agent 34 的初始资源申请量为: 50.0
agent 35 的初始资源申请量为: 50.0
agent 36 的初始资源申请量为: 50.0
agent 37 的初始资源申请量为: 50.0
ag

39 次迭代中的结果：
agent 1 在 39 次迭代中的资源申请量为: 48.44257034141743
agent 2 在 39 次迭代中的资源申请量为: 61.02821292677224
agent 3 在 39 次迭代中的资源申请量为: 43.84344441889768
agent 4 在 39 次迭代中的资源申请量为: 64.53986864738874
agent 5 在 39 次迭代中的资源申请量为: 65.33986864738873
agent 6 在 39 次迭代中的资源申请量为: 72.11385551211634
agent 7 在 39 次迭代中的资源申请量为: 39.502029216908426
agent 8 在 39 次迭代中的资源申请量为: 37.756927756063
agent 9 在 39 次迭代中的资源申请量为: 43.14344441889769
agent 10 在 39 次迭代中的资源申请量为: 55.833735009327974
agent 11 在 39 次迭代中的资源申请量为: 45.52115969507881
agent 12 在 39 次迭代中的资源申请量为: 49.342570341417435
agent 13 在 39 次迭代中的资源申请量为: 65.83986864738873
agent 14 在 39 次迭代中的资源申请量为: 55.23373500932797
agent 15 在 39 次迭代中的资源申请量为: 43.54344441889768
agent 16 在 39 次迭代中的资源申请量为: 43.74344441889768
agent 17 在 39 次迭代中的资源申请量为: 44.92115969507881
agent 18 在 39 次迭代中的资源申请量为: 46.52115969507881
agent 19 在 39 次迭代中的资源申请量为: 41.352141951181125
agent 20 在 39 次迭代中的资源申请量为: 55.73373500932798
agent 21 在 39 次迭代中的资源申请量为: 38.80202921690843
agent 22 在 39 次迭代中的资源申请量为: 39.10202921690843
agent

59 次迭代中的结果：
agent 1 在 59 次迭代中的资源申请量为: 48.4425703414172
agent 2 在 59 次迭代中的资源申请量为: 61.02821292677151
agent 3 在 59 次迭代中的资源申请量为: 43.843444418897526
agent 4 在 59 次迭代中的资源申请量为: 64.53986864738707
agent 5 在 59 次迭代中的资源申请量为: 65.33986864738706
agent 6 在 59 次迭代中的资源申请量为: 72.11385551212577
agent 7 在 59 次迭代中的资源申请量为: 39.5020292169083
agent 8 在 59 次迭代中的资源申请量为: 37.756927756062886
agent 9 在 59 次迭代中的资源申请量为: 43.1434444188975
agent 10 在 59 次迭代中的资源申请量为: 55.8337350093275
agent 11 在 59 次迭代中的资源申请量为: 45.521159695078595
agent 12 在 59 次迭代中的资源申请量为: 49.34257034141717
agent 13 在 59 次迭代中的资源申请量为: 65.83986864738704
agent 14 在 59 次迭代中的资源申请量为: 55.233735009327496
agent 15 在 59 次迭代中的资源申请量为: 43.5434444188975
agent 16 在 59 次迭代中的资源申请量为: 43.743444418897504
agent 17 在 59 次迭代中的资源申请量为: 44.92115969507859
agent 18 在 59 次迭代中的资源申请量为: 46.521159695078595
agent 19 在 59 次迭代中的资源申请量为: 41.35214195118097
agent 20 在 59 次迭代中的资源申请量为: 55.7337350093275
agent 21 在 59 次迭代中的资源申请量为: 38.802029216908295
agent 22 在 59 次迭代中的资源申请量为: 39.102029216908285
agent

79 次迭代中的结果：
agent 1 在 79 次迭代中的资源申请量为: 48.4425703414172
agent 2 在 79 次迭代中的资源申请量为: 61.02821292677151
agent 3 在 79 次迭代中的资源申请量为: 43.843444418897526
agent 4 在 79 次迭代中的资源申请量为: 64.53986864738707
agent 5 在 79 次迭代中的资源申请量为: 65.33986864738706
agent 6 在 79 次迭代中的资源申请量为: 72.11385551212577
agent 7 在 79 次迭代中的资源申请量为: 39.5020292169083
agent 8 在 79 次迭代中的资源申请量为: 37.756927756062886
agent 9 在 79 次迭代中的资源申请量为: 43.1434444188975
agent 10 在 79 次迭代中的资源申请量为: 55.8337350093275
agent 11 在 79 次迭代中的资源申请量为: 45.521159695078595
agent 12 在 79 次迭代中的资源申请量为: 49.34257034141717
agent 13 在 79 次迭代中的资源申请量为: 65.83986864738704
agent 14 在 79 次迭代中的资源申请量为: 55.233735009327496
agent 15 在 79 次迭代中的资源申请量为: 43.5434444188975
agent 16 在 79 次迭代中的资源申请量为: 43.743444418897504
agent 17 在 79 次迭代中的资源申请量为: 44.92115969507859
agent 18 在 79 次迭代中的资源申请量为: 46.521159695078595
agent 19 在 79 次迭代中的资源申请量为: 41.35214195118097
agent 20 在 79 次迭代中的资源申请量为: 55.7337350093275
agent 21 在 79 次迭代中的资源申请量为: 38.802029216908295
agent 22 在 79 次迭代中的资源申请量为: 39.102029216908285
agent

99 次迭代中的结果：
agent 1 在 99 次迭代中的资源申请量为: 48.4425703414172
agent 2 在 99 次迭代中的资源申请量为: 61.02821292677151
agent 3 在 99 次迭代中的资源申请量为: 43.843444418897526
agent 4 在 99 次迭代中的资源申请量为: 64.53986864738707
agent 5 在 99 次迭代中的资源申请量为: 65.33986864738706
agent 6 在 99 次迭代中的资源申请量为: 72.11385551212577
agent 7 在 99 次迭代中的资源申请量为: 39.5020292169083
agent 8 在 99 次迭代中的资源申请量为: 37.756927756062886
agent 9 在 99 次迭代中的资源申请量为: 43.1434444188975
agent 10 在 99 次迭代中的资源申请量为: 55.8337350093275
agent 11 在 99 次迭代中的资源申请量为: 45.521159695078595
agent 12 在 99 次迭代中的资源申请量为: 49.34257034141717
agent 13 在 99 次迭代中的资源申请量为: 65.83986864738704
agent 14 在 99 次迭代中的资源申请量为: 55.233735009327496
agent 15 在 99 次迭代中的资源申请量为: 43.5434444188975
agent 16 在 99 次迭代中的资源申请量为: 43.743444418897504
agent 17 在 99 次迭代中的资源申请量为: 44.92115969507859
agent 18 在 99 次迭代中的资源申请量为: 46.521159695078595
agent 19 在 99 次迭代中的资源申请量为: 41.35214195118097
agent 20 在 99 次迭代中的资源申请量为: 55.7337350093275
agent 21 在 99 次迭代中的资源申请量为: 38.802029216908295
agent 22 在 99 次迭代中的资源申请量为: 39.102029216908285
agent

...

1899 次迭代中的结果：
agent 1 在 1899 次迭代中的资源申请量为: 48.4425703414172
agent 2 在 1899 次迭代中的资源申请量为: 61.02821292677151
agent 3 在 1899 次迭代中的资源申请量为: 43.843444418897526
agent 4 在 1899 次迭代中的资源申请量为: 64.53986864738707
agent 5 在 1899 次迭代中的资源申请量为: 65.33986864738706
agent 6 在 1899 次迭代中的资源申请量为: 72.11385551212577
agent 7 在 1899 次迭代中的资源申请量为: 39.5020292169083
agent 8 在 1899 次迭代中的资源申请量为: 37.756927756062886
agent 9 在 1899 次迭代中的资源申请量为: 43.1434444188975
agent 10 在 1899 次迭代中的资源申请量为: 55.8337350093275
agent 11 在 1899 次迭代中的资源申请量为: 45.521159695078595
agent 12 在 1899 次迭代中的资源申请量为: 49.34257034141717
agent 13 在 1899 次迭代中的资源申请量为: 65.83986864738704
agent 14 在 1899 次迭代中的资源申请量为: 55.233735009327496
agent 15 在 1899 次迭代中的资源申请量为: 43.5434444188975
agent 16 在 1899 次迭代中的资源申请量为: 43.743444418897504
agent 17 在 1899 次迭代中的资源申请量为: 44.92115969507859
agent 18 在 1899 次迭代中的资源申请量为: 46.521159695078595
agent 19 在 1899 次迭代中的资源申请量为: 41.35214195118097
agent 20 在 1899 次迭代中的资源申请量为: 55.7337350093275
agent 21 在 1899 次迭代中的资源申请量为: 38.802029216908295
agent 2

1929 次迭代中的结果：
agent 1 在 1929 次迭代中的资源申请量为: 48.4425703414172
agent 2 在 1929 次迭代中的资源申请量为: 61.02821292677151
agent 3 在 1929 次迭代中的资源申请量为: 43.843444418897526
agent 4 在 1929 次迭代中的资源申请量为: 64.53986864738707
agent 5 在 1929 次迭代中的资源申请量为: 65.33986864738706
agent 6 在 1929 次迭代中的资源申请量为: 72.11385551212577
agent 7 在 1929 次迭代中的资源申请量为: 39.5020292169083
agent 8 在 1929 次迭代中的资源申请量为: 37.756927756062886
agent 9 在 1929 次迭代中的资源申请量为: 43.1434444188975
agent 10 在 1929 次迭代中的资源申请量为: 55.8337350093275
agent 11 在 1929 次迭代中的资源申请量为: 45.521159695078595
agent 12 在 1929 次迭代中的资源申请量为: 49.34257034141717
agent 13 在 1929 次迭代中的资源申请量为: 65.83986864738704
agent 14 在 1929 次迭代中的资源申请量为: 55.233735009327496
agent 15 在 1929 次迭代中的资源申请量为: 43.5434444188975
agent 16 在 1929 次迭代中的资源申请量为: 43.743444418897504
agent 17 在 1929 次迭代中的资源申请量为: 44.92115969507859
agent 18 在 1929 次迭代中的资源申请量为: 46.521159695078595
agent 19 在 1929 次迭代中的资源申请量为: 41.35214195118097
agent 20 在 1929 次迭代中的资源申请量为: 55.7337350093275
agent 21 在 1929 次迭代中的资源申请量为: 38.802029216908295
agent 2

1949 次迭代中的结果：
agent 1 在 1949 次迭代中的资源申请量为: 48.4425703414172
agent 2 在 1949 次迭代中的资源申请量为: 61.02821292677151
agent 3 在 1949 次迭代中的资源申请量为: 43.843444418897526
agent 4 在 1949 次迭代中的资源申请量为: 64.53986864738707
agent 5 在 1949 次迭代中的资源申请量为: 65.33986864738706
agent 6 在 1949 次迭代中的资源申请量为: 72.11385551212577
agent 7 在 1949 次迭代中的资源申请量为: 39.5020292169083
agent 8 在 1949 次迭代中的资源申请量为: 37.756927756062886
agent 9 在 1949 次迭代中的资源申请量为: 43.1434444188975
agent 10 在 1949 次迭代中的资源申请量为: 55.8337350093275
agent 11 在 1949 次迭代中的资源申请量为: 45.521159695078595
agent 12 在 1949 次迭代中的资源申请量为: 49.34257034141717
agent 13 在 1949 次迭代中的资源申请量为: 65.83986864738704
agent 14 在 1949 次迭代中的资源申请量为: 55.233735009327496
agent 15 在 1949 次迭代中的资源申请量为: 43.5434444188975
agent 16 在 1949 次迭代中的资源申请量为: 43.743444418897504
agent 17 在 1949 次迭代中的资源申请量为: 44.92115969507859
agent 18 在 1949 次迭代中的资源申请量为: 46.521159695078595
agent 19 在 1949 次迭代中的资源申请量为: 41.35214195118097
agent 20 在 1949 次迭代中的资源申请量为: 55.7337350093275
agent 21 在 1949 次迭代中的资源申请量为: 38.802029216908295
agent 2

1969 次迭代中的结果：
agent 1 在 1969 次迭代中的资源申请量为: 48.4425703414172
agent 2 在 1969 次迭代中的资源申请量为: 61.02821292677151
agent 3 在 1969 次迭代中的资源申请量为: 43.843444418897526
agent 4 在 1969 次迭代中的资源申请量为: 64.53986864738707
agent 5 在 1969 次迭代中的资源申请量为: 65.33986864738706
agent 6 在 1969 次迭代中的资源申请量为: 72.11385551212577
agent 7 在 1969 次迭代中的资源申请量为: 39.5020292169083
agent 8 在 1969 次迭代中的资源申请量为: 37.756927756062886
agent 9 在 1969 次迭代中的资源申请量为: 43.1434444188975
agent 10 在 1969 次迭代中的资源申请量为: 55.8337350093275
agent 11 在 1969 次迭代中的资源申请量为: 45.521159695078595
agent 12 在 1969 次迭代中的资源申请量为: 49.34257034141717
agent 13 在 1969 次迭代中的资源申请量为: 65.83986864738704
agent 14 在 1969 次迭代中的资源申请量为: 55.233735009327496
agent 15 在 1969 次迭代中的资源申请量为: 43.5434444188975
agent 16 在 1969 次迭代中的资源申请量为: 43.743444418897504
agent 17 在 1969 次迭代中的资源申请量为: 44.92115969507859
agent 18 在 1969 次迭代中的资源申请量为: 46.521159695078595
agent 19 在 1969 次迭代中的资源申请量为: 41.35214195118097
agent 20 在 1969 次迭代中的资源申请量为: 55.7337350093275
agent 21 在 1969 次迭代中的资源申请量为: 38.802029216908295
agent 2

1989 次迭代中的结果：
agent 1 在 1989 次迭代中的资源申请量为: 48.4425703414172
agent 2 在 1989 次迭代中的资源申请量为: 61.02821292677151
agent 3 在 1989 次迭代中的资源申请量为: 43.843444418897526
agent 4 在 1989 次迭代中的资源申请量为: 64.53986864738707
agent 5 在 1989 次迭代中的资源申请量为: 65.33986864738706
agent 6 在 1989 次迭代中的资源申请量为: 72.11385551212577
agent 7 在 1989 次迭代中的资源申请量为: 39.5020292169083
agent 8 在 1989 次迭代中的资源申请量为: 37.756927756062886
agent 9 在 1989 次迭代中的资源申请量为: 43.1434444188975
agent 10 在 1989 次迭代中的资源申请量为: 55.8337350093275
agent 11 在 1989 次迭代中的资源申请量为: 45.521159695078595
agent 12 在 1989 次迭代中的资源申请量为: 49.34257034141717
agent 13 在 1989 次迭代中的资源申请量为: 65.83986864738704
agent 14 在 1989 次迭代中的资源申请量为: 55.233735009327496
agent 15 在 1989 次迭代中的资源申请量为: 43.5434444188975
agent 16 在 1989 次迭代中的资源申请量为: 43.743444418897504
agent 17 在 1989 次迭代中的资源申请量为: 44.92115969507859
agent 18 在 1989 次迭代中的资源申请量为: 46.521159695078595
agent 19 在 1989 次迭代中的资源申请量为: 41.35214195118097
agent 20 在 1989 次迭代中的资源申请量为: 55.7337350093275
agent 21 在 1989 次迭代中的资源申请量为: 38.802029216908295
agent 2

In [6]:
np.savetxt('../data/03-robust_agent_resource_quantity_large_value_Gaussian_bias(500,10).txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/03-robust_sum_cost_large_value_Gaussian_bias(500,10).txt',sum_cost,fmt='%0.8f')#用户成本总和 
np.savetxt('../data/03-robust_sum_resource_large_value_Gaussian_bias(500,10).txt',sum_resource,fmt='%0.8f')#用户资源总和 
np.savetxt('../data/03-robust_agent_gradient_large_value_Gaussian_bias(500,10).txt',agent_gradient,fmt='%0.8f')#用户迭代梯度 